In [1]:
import sys
sys.path.append('../../Modules/')
sys.path.append('../../Modules/Processors from Prof')
from Packages import *
from My_Json_processor import *

# Import processors from Prof
from ipynb.fs.full.Utilities import *
from ipynb.fs.full.Json_Processor import *
from ipynb.fs.full.CSV_Processor import *

### My Testing

In [2]:
boolean_df = Big_Bench_Json_Processor_my('Big Bench Hard', 'boolean_expressions.json').convert_df()

In [3]:
boolean_df.head(20)

,input,target
0,not ( True ) and ( True ) is,False
1,True and not not ( not False ) is,True
2,not True or False or ( False ) is,False
3,False or not ( True ) and False is,False
4,True or not False and True and False is,True
5,False or not not not False and True is,True
6,not True and ( False or True ) is,False
7,True and not False or ( True ) is,True
8,not True or ( False and True ) is,False
9,not True or ( True or False ) is,True


### Professor Testing

In [4]:
contents = os.listdir('../../Big Bench Hard/data')

In [5]:
sample_lst = [file for file in os.listdir('../../Big Bench Hard/data') if file.endswith("json")]

In [6]:
os.listdir('../../Big Bench Hard/data')

['object_counting.json',
 'temporal_sequences.json',
 'tracking_shuffled_objects_five_objects.json',
 'word_sorting.json',
 'salient_translation_error_detection.json',
 'penguins_in_a_table.json',
 'snarks.json',
 'ruin_names.json',
 'web_of_lies.json',
 'navigate.json',
 'hyperbaton.json',
 'dyck_languages.json',
 'tracking_shuffled_objects_three_objects.json',
 'formal_fallacies.json',
 'tracking_shuffled_objects_seven_objects.json',
 'causal_judgement.json',
 'sports_understanding.json',
 'logical_deduction_five_objects.json',
 'movie_recommendation.json',
 '.ipynb_checkpoints',
 'logical_deduction_three_objects.json',
 'multistep_arithmetic_two.json',
 'boolean_expressions.json',
 'geometric_shapes.json',
 'disambiguation_qa.json',
 'logical_deduction_seven_objects.json',
 'reasoning_about_colored_objects.json']

In [7]:
output_file_path = 'output.json'